In [1]:
from src.ir_system import IrSystem
from src.movie_description import MovieDescription, create_corpus

# Update the dictionary
Boolean information retrieval system project focused on the update of the collection allowing for addition and deletion of documents.

### Creazione del sitema di IR

Step 1: Creare il corpus di documenti

In [2]:
corpus = create_corpus('data/movie.metadata.tsv', 'data/plot_summaries.txt')
total_docs = len(corpus)
total_docs

42204

Step 2: Creare il sistema IR (inverted index ed inverted index biword)

In [3]:
ir = IrSystem.create_system(corpus)

100%|██████████| 42204/42204 [00:30<00:00, 1393.85it/s]


### Esempi di query
#### 1. Query semplice

In [4]:
ir.query("yoda")

['3731: Star Wars Episode V: The Empire Strikes Back',
 '7833: Star Wars Episode II: Attack of the Clones',
 '9268: George Lucas in Love',
 '18527: Something, Something, Something Dark Side',
 '23373: Return of the Ewok',
 '24906: Aliens in the Wild, Wild West',
 '27247: Star Wars Episode III: Revenge of the Sith',
 '31514: Star Wars Episode VI: Return of the Jedi',
 '33732: Star Wars: The Clone Wars',
 "37962: Gulliver's Travels",
 '39161: Lego Star Wars: The Quest for R2-D2',
 "40974: It's a Trap!",
 '42160: LEGO Star Wars: Revenge of the Brick']

#### 2. query logiche

In [5]:
ir.query("(yoda AND NOT wars) OR skywalker AND luke")

['3731: Star Wars Episode V: The Empire Strikes Back',
 '18527: Something, Something, Something Dark Side',
 '23373: Return of the Ewok',
 '31514: Star Wars Episode VI: Return of the Jedi',
 "40974: It's a Trap!"]

In [6]:
ir.query("parrot AND spaceship")

[]

#### 3. Phrase query (ritorna i documenti che contengono esattamente la sequenza di parole della query)

In [7]:
ir.phrase_query("luke skywalker")

['3731: Star Wars Episode V: The Empire Strikes Back',
 '6984: Imaginationland Episode II',
 '8341: Star Wars Episode IV: A New Hope',
 '8867: Wishology',
 '12767: Lego Star Wars: Bombad Bounty',
 '18527: Something, Something, Something Dark Side',
 '18532: The Star Wars Holiday Special',
 '23373: Return of the Ewok',
 '24258: The Making of Star Wars',
 '31514: Star Wars Episode VI: Return of the Jedi',
 "40974: It's a Trap!",
 '41068: Robot Chicken: Star Wars Episode II']

### Aggiunta di un documento

In [8]:
# prima dell'aggiunta
ir.phrase_query("sette amici")

[]

In [9]:
new_title = "Perfetti sconosciuti"
new_description = "Film italiano diretto da Paolo Genovese. Racconta di sette amici che, durante una cena, decidono di condividere messaggi e chiamate ricevuti sul cellulare. Il gioco fa emergere segreti nascosti, mettendo a dura prova le loro relazioni."
new_movie = MovieDescription(new_title, new_description)
ir.add_docs([new_movie])

100%|██████████| 1/1 [00:00<00:00, 7345.54it/s]


In [10]:
# dopo l'aggiunta
ir.phrase_query("sette amici")

['42204: Perfetti sconosciuti']

### Rimozione di documenti

In [11]:
# prima della rimozione
ir.query("yoda")

['3731: Star Wars Episode V: The Empire Strikes Back',
 '7833: Star Wars Episode II: Attack of the Clones',
 '9268: George Lucas in Love',
 '18527: Something, Something, Something Dark Side',
 '23373: Return of the Ewok',
 '24906: Aliens in the Wild, Wild West',
 '27247: Star Wars Episode III: Revenge of the Sith',
 '31514: Star Wars Episode VI: Return of the Jedi',
 '33732: Star Wars: The Clone Wars',
 "37962: Gulliver's Travels",
 '39161: Lego Star Wars: The Quest for R2-D2',
 "40974: It's a Trap!",
 '42160: LEGO Star Wars: Revenge of the Brick']

In [12]:
ir.delete_docs([i for i in range(20000, 39999)])

In [13]:
# dopo la rimozione
ir.query("yoda")

['3731: Star Wars Episode V: The Empire Strikes Back',
 '7833: Star Wars Episode II: Attack of the Clones',
 '9268: George Lucas in Love',
 '18527: Something, Something, Something Dark Side',
 "40974: It's a Trap!",
 '42160: LEGO Star Wars: Revenge of the Brick']

### Test aggiunta e rimozione su tutto il corpus
Per testare, si divide il corpus in 3 parti, A, B, e C. Inizialmente l'indice contiene solo A e B, poi C deve essere aggiunta e C.

In [14]:
len_AB = (2 * total_docs) // 3
len_C = total_docs - len_AB
print(f"tot: {total_docs},\nA&B: {len_AB},\nC: {len_C}")

tot: 42204,
A&B: 28136,
C: 14068


In [15]:
A_end = len_AB // 2
B_end = len_AB

# parte A
corpus_A = corpus[:A_end]
# parte B
corpus_B = corpus[A_end:B_end]
# parte C
corpus_C = corpus[B_end:]

#### 1. crea il sistema (IrSystem) iniziale solo con A+B

In [16]:
corpus_init = corpus_A + corpus_B

In [17]:
ir_test = IrSystem.create_system(corpus_init)
print(f"Corpus iniziale: {len(ir_test._corpus)} documenti")

100%|██████████| 28136/28136 [00:28<00:00, 979.88it/s] 


Corpus iniziale: 28136 documenti


Query di test:

In [18]:
ir_test.query("sette amici")

['36: Amici miei', '20304: Earthquake']

#### 2. aggiungi la parte C usando la funzione `IrSystem.add_docs()`

In [19]:
ir_test.add_docs(corpus_C)

100%|██████████| 14069/14069 [00:17<00:00, 813.24it/s] 


In [20]:
print(f"Dopo add_docs(C):\n- dimensione indice principale: {len(ir_test._index)}\n"
      f"- dimensione aux index: {len(ir_test._aux_idx) if ir_test._aux_idx else 0}\n"
      f"- corpus attuale: {len(ir_test._corpus)} documenti")

Dopo add_docs(C):
- dimensione indice principale: 111868
- dimensione aux index: 73245
- corpus attuale: 42205 documenti


Ora la query di esempio ritorna un risultato in più:

In [21]:
ir_test.query("sette amici")

['36: Amici miei', '20304: Earthquake', '42204: Perfetti sconosciuti']

Oss. il sistema effettua il marge dei due indici quando la dimensione dell'indice ausiliario raggiunge `max_size_aux = 80000`. In questo caso tale numero non viene raggiunto, quindi chiamiamo la funzione `_merge_idx()` per forzare il merge e vedere come funziona il sistema:

In [22]:
ir_test._merge_idx()
print(f"Dopo il merge degli indici:\n"
      f"- dimensione indice principale: {len(ir_test._index)}\n"
      f"- dimensione aux index: {len(ir_test._aux_idx) if ir_test._aux_idx else 0}\n")

Dopo il merge degli indici:
- dimensione indice principale: 144529
- dimensione aux index: 0



#### 3. elimina la parte B usando la funzione `IrSystem.delete_docs()`

In [23]:
# I docID di corpus_B partono dopo corpus_A, quindi da len(corpus_A)
start_docID_B = len(corpus_A)
end_docID_B = start_docID_B + len(corpus_B)
docIDs_B = list(range(start_docID_B, end_docID_B))

Oss. `delete_docs()` segna nell'invalid vector i docID dei documenti da eliminare. La vera eliminazione viene fatta nel `_merge_idx()`.

In [24]:
ir_test.delete_docs(docIDs_B)
ir_test._merge_idx()

In [25]:
print(f"Dopo delete_docs(B):\n- documenti marcati come cancellati: "
      f"{sum(ir_test._invalid_vec)} su {len(ir_test._invalid_vec)} totali.\n"
      f"- dimensione indice: {len(ir_test._index)}\n"
      f"- corpus attuale: {len(ir_test)} documenti")

Dopo delete_docs(B):
- documenti marcati come cancellati: 14068 su 42205 totali.
- dimensione indice: 112054
- corpus attuale: 28137 documenti


Ora la query di esempio ritorna un risultato in meno:

In [26]:
ir_test.query("sette amici")

['36: Amici miei', '42204: Perfetti sconosciuti']